##  SQL

### Очень усердные ученики.

Условие

Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной".

Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин.

Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов.

In [197]:
# подгружаем необходимые библиотеки
#pip install pandahouse  # если библиотека ещё не установлена
import pandahouse as ph

In [198]:
!pip install pandahouse

In [199]:
# Для того , чтобы сканектиться с базой данных SQL 

In [200]:
connection_default = {'host': 'http://clickhouse.beslan.pro:8080',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

In [201]:
# Посмотрим на таблицу в целом

In [202]:
#пишем запрос, и получаем данные из clickhouse в pandas dataframe
query = """
SELECT  
    st_id,
    toDateTime(timest) as timest,
    correct,
    subject 
FROM default.peas

"""
df = ph.read_clickhouse(query, connection=connection_default)
df.head()

,st_id,timest,correct,subject
0,100379,2021-10-30 13:32:29,1,Theory of probability
1,100379,2021-10-30 14:11:19,0,Vizualization
2,100379,2021-10-30 15:54:22,1,Theory of probability
3,100379,2021-10-30 16:44:50,1,Vizualization
4,100379,2021-10-30 17:15:05,1,Theory of probability


In [203]:
# Посмотрим на усердных студентов 

In [204]:
query = """

SELECT  
    COUNT(st_id) AS count_st_id
FROM default.peas
WHERE  correct = 1
GROUP BY  st_id 
HAVING SUM(correct) >= 20
ORDER BY SUM(correct) DESC
"""
st_id = ph.read_clickhouse(query, connection=connection_default)
st_id.head()

,count_st_id
0,126
1,121
2,111
3,81
4,79


In [205]:
# количестве очень усердных студентов одним запросом.

In [206]:
#пишем запрос, и получаем данные из clickhouse в pandas dataframe
# Так как в таблице данные только за 2021 год октябрь, нельзя вывести значения на текущий месяц. 
# поэтому запишем конректно месяц
query = """

SELECT COUNT(c.count_st_id)
FROM(
SELECT  
    COUNT(st_id) AS count_st_id
FROM default.peas
WHERE  correct = 1 AND  MONTH(timest) = '10'   --MONTH(timest) = MONTH(NOW()) -- функция для текущего месяца
GROUP BY  st_id                                                              -- но в таблице нет этих данных
HAVING SUM(correct) >= 20
ORDER BY SUM(correct) DESC) AS c

"""
count_id = ph.read_clickhouse(query, connection=connection_default)
count_id 

,count(count_st_id)
0,136


### 2.2 Оптимизация воронки

2.2 Условие

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:


- ARPU (Average Revenue Per User ) - отношение выручки к общему количеству пользователей.


- ARPAU (Average Revenue Per Active User) - отношение выручки к количеству активных пользователей.


- CR в покупку - отношение количества купивших пользователей к общему количеству пользователей.


- СR активного пользователя в покупку - отношение количества купивших активных пользователей к общему количеству активных пользователей


- CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике - отношение количества купивших активных в математике пользователей к общему количеству активных в математике пользователей.


- ARPU считается относительно всех пользователей, попавших в группы.


- Активным считается пользователь, за все время решивший больше 10 задач правильно в любых дисциплинах.


- Активным по математике считается пользователь, за все время решивший 2 или больше задач правильно по математике.

Для формирования запроса нам потребуются данные из таблиц, на основе которых будем делать расчет необходимых метрик
  
1. Из таблицы **peas**  запросим список пользвателей . 1 - активные пользователи( те которые решили больше 10 задач ) 

 
    
2. Так же из таблицы **peas** отберем пользователей с активностью по математике 
1 - активные 


3. таблицу  **studs** возьмем полностью. Из нее возьмем сведенья о тестовой и контрольной группах


4. Из таблицы **final_project_check** возьмем пользователей , присвоив 0 - тем, кто купил любой предмет 1 - математику

    
5. Объединим все таблицы , для подсчета нужных метрик


In [207]:
# Из таблицы peas получим  новую колонку  пользователей с отметками  "1" об активности в целом. Обернем
# запрос в подзапрос

(SELECT st_id, 
       1 AS active_total --# присваиваем 1 - активным     
FROM default.peas 
    GROUP BY  st_id 
    HAVING SUM(correct) > 10
    )  AS a

In [ ]:
# Так же из таблицы peas отберем пользователей с активностью по математике 1 - активные 

( SELECT st_id, 
         1 AS active_subj -- # присваиваем 1 - активным по мат 

    FROM default.peas 
    WHERE subject = 'Math'
    GROUP BY  st_id 
    HAVING SUM(correct) > 1)  AS b


In [ ]:
# таблицу studs возьмем полностью. Из нее возьмем сведенья о тестовой и контрольной группах
    studs     


In [ ]:
 # Из таблицы final_project_check возьмем пользователей , присвоив 0 - тем, кто купил любой предмет 1 - математику

(SELECT st_id,
     SUM(money) AS money,
     1 AS sale_subject,
     MAX(IF(subject = 'Math', 1, 0 )) AS sale_math
FROM default.final_project_check
GROUP BY st_id) AS c

### Объединим все 4 подзапроса и посчитаем  метрики

В общем запросе вывода данных укажем расчет метрик. В блоке FROM  - объединенные таблицы 
Каждому запросу дадим имя ( a,b,c ). Сгруппируем по группам (test/control)

При этом первой возьмем таблицу studs, так как в ней все данные о студентах. деленные по группам
Затем к этой таблице будет присоединять остальные подзапросы.


In [ ]:
connection_default = {'host': 'http://clickhouse.beslan.pro:8080',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

In [209]:
#пишем запрос, и получаем данные из clickhouse в pandas dataframe
query = """

SELECT
    studs.test_grp, 
    ROUND(SUM(c.money) / COUNT(studs.st_id),2) AS ARPU,-- #сумма покупок / всех пользователей
    ROUND(SUM(c.money * a.active_total) / SUM(a.active_total),2) AS ARPAU,-- #сумма покупок / всех активных пользователей
    ROUND(SUM(c.sale_subject) / COUNT(studs.st_id),2) AS CR,-- #количество покупок / количесвто пользователей
    ROUND(SUM( a.active_total * c.sale_subject) / SUM(a.active_total),2) AS CR_active, --#количесвто
   -- #активных купивших/ количесвто активных пользователей
            
    ROUND(SUM(c.sale_math * b.active_subj) / SUM(b.active_subj),2) AS CR_math -- #количесвто активных купивших
  --  #по математике / количесвто активных по всем предметам     
 
 
FROM
    
default.studs 

LEFT JOIN 
(SELECT st_id,
     SUM(money) AS money,
     1 AS sale_subject,
     MAX(IF(subject = 'Math', 1, 0 )) AS sale_math
FROM default.final_project_check
GROUP BY st_id) AS c
    
ON studs.st_id = c.st_id

LEFT JOIN

(SELECT st_id, 
       1 AS active_total --# присваиваем 1 - активным     
FROM default.peas 
    GROUP BY  st_id 
    HAVING SUM(correct) > 10
    )  AS a
    
    ON studs.st_id = a.st_id 
    
LEFT JOIN
    
( SELECT st_id, 
         1 AS active_subj -- # присваиваем 1 - активным по мат 

    FROM default.peas 
    WHERE subject = 'Math'
    GROUP BY  st_id 
    HAVING SUM(correct) > 1)  AS b
    
ON  a.st_id =  b.st_id  
    
GROUP BY studs.test_grp

"""

metrix = ph.read_clickhouse(query, connection=connection_default)
metrix 

,studs.test_grp,ARPU,ARPAU,CR,CR_active,CR_math
0,control,4540.98,10393.70,0.05,0.11,0.07
1,pilot,11508.47,29739.58,0.11,0.26,0.14
